
- overall goal:
    - specific ROI targetting w/ minimal campaigns
- what this means for accnt structure
    - for many variables we must split campaigns to target ROI
    - want to capture greatest amt of rps variation w/ 
        fewest # of campaigns
- 2 ways of approaching this
    1. minimize rps variation w/in a campaign
        - i.e. after campaign split want to minimize rps variance w/in campaigns
        => i actually think this is eq to decision tree regression w/ split criterion MSE
        https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
        - tried this out a little - but the computation was intensive which made it slow going
    2. maximize rps variation outside campaigns
    - i.e. after campaign split want to minimize rps variance w/in campaigns
        => i actually think this is eq to decision tree regression w/ split criterion MSE
        https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
    - mostly focused on this approach

- approaches I tried
1. rank splits by their correlation/covaraince/MI w/ `agg_rps`
    - tried ANOVA,pt.biserial,MI - had most success w/ MI
    - was dificult to compute b/c couldnt find methods that accepted sample weight
    - MI approach was promising but didnt go down that route
2. rank splits by intra-split agg_rps variance
    - had most success w/ this
    - tested out an approahc where I cluster the split on rps - had good results
    - think this is the mtd to use going forward
    TODO:
    - test fitting on general traffic sources?
    - test clustering on multiple days of rps data - 
        or rolling rps data or something
    - try out more granular aggs
    - is there some kind of metric which measures total variance
        and computes how much of that variance is captured by a split?

3. fit decision tree on rps data 
    - MSE criterion is apparently the same as minimizng inter split variance
    - computationally intensive
    - not 100% clear how to go from tree to campaign structure


- can do this by
    1. choosing what vars or tuples of vars to split campaigns on
    2. grouping similar buckets w/in those splits
- want to measure correlation/dependence of categorical split vars w/ cont rps
    - cat,cat metrics w/ binned rps
        chi
    - cont,cont metrcs w/ 1 hot encoded split vars
    - cat,cont metrics
- correlation metrics
    - MI:
        - sklearn.feature_selection.mi_regression
        - would need AFAICT to rresample input arrays
    - ANOVA:
        - scipy.stats.f_oneway
    - pt biserial
        - needs binary vars tho
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pointbiserialr.html
        - https://towardsdatascience.com/point-biserial-correlation-with-python-f7cd591bd3b1
- want most variation of rps w/ fewest camapaigns
- dont want to group similar buckets together - think its too complicated


In [ ]:
from utils.env import load_env_from_aws
load_env_from_aws()

import datetime
from notebooks.aduriseti_shared.utils import *

TABOOLA = "TABOOLA"
MEDIA_ALPHA = "MEDIAALPHA"
BING = "BING"
U65 = "HEALTH"
O65 = 'MEDICARE'

NOW = datetime.datetime.now()
DAY = datetime.timedelta(days=1)

campaign_split_fields = dict(
    # traffic_source = ["traffic_source"],
    browser = ["browser"],
    operating_system = ["operating_system"],
    device = ["device"],
    # channel = ["channel"],
    # domain = ["domain"],
    product = ["product"],
    # keyword = ["keyword"],
    # campaign_id = ["campaign_id"],
    # landing_page = ["landing_page"],
    TOD = ["dayofweek","hourofday"],
    dma = ["dma"],
    state =["state",],
    location = ["state","dma"],
    
    dma_os=["dma", "operating_system"],
    dma_device=["dma", "device", ],
    dma_os_device=["dma", "operating_system", "device"],

    state_os=["state", "operating_system"],
    state_device=["state", "device", ],
    state_os_device=["state", "operating_system", "device"],

    location_os = ["state", "dma", "operating_system"],
    location_device=["state", "dma", "device", ],
    location_os_device = ["state", "dma", "operating_system","device"],
)

taboola_val_map = {
    "device": {
        'DESKTOP': 'DESK',
        'MOBILE': 'PHON',
        'TABLET': 'TBLT',
    },
    "operating_system": {
        'Linux armv7l': "Linux",
        'Linux armv8l': "Linux",
        'Linux x86_64': "Linux",
        'MacIntel': 'Mac OS X',
        'Win32': "Windows",
        'iPad': "iPadOS",
        'iPhone': "iOS",
        '': None,
        'ARM': None,
        'Android': 'Android',
        'Linux aarch64': "Linux",
        'Win64': "Windows",
        'Linux armv7': "Linux",
        'Linux i686': "Linux",
        'Windows': "Windows",
    }
}

def translate_taboola_vals(df):
    index_cols = df.index.names
    df = df.reset_index()
    for c in df.columns:
        if c in taboola_val_map:
            df[c] = df[c].apply(taboola_val_map[c].__getitem__)
    df_bkp = df
    df = df \
        .groupby(index_cols) \
        .agg({
            "sessions": sum,
            "num_leads": sum,
            "lps_avg": get_wavg_by(df,"sessions"),
            "rpl_avg": get_wavg_by(df,"sessions"),
            "rps_avg": get_wavg_by(df,"sessions"),
        })
    df["int_ix"] = range(len(df))
    df_bkp_wavg = wavg(df_bkp[["lps_avg","rpl_avg","rps_avg"]],
                        df_bkp["sessions"].values.reshape(-1, 1))
    df_wavg = wavg(df[["lps_avg","rpl_avg","rps_avg"]],
                    df["sessions"].values.reshape(-1, 1))
    assert all((df_bkp_wavg - df_wavg).abs() < 1e-2), (df_bkp_wavg,df_wavg)
    return df

import pprint
from IPython.display import display as ipydisp    
import pandas as pd
from models.utils import wavg
import collections
import itertools

def wvar(V,W):
    mu = wavg(V,W)
    var = wavg((V - mu)**2,W)
    return var
def wstd(V,W):
    return wvar(V,W)**0.5
def get_wavg_by(df, col):
    def wavg_by(V):
        return wavg(V, W=df.loc[V.index, col])
    return wavg_by

def get_wthresh(W,p):
    W = rps_df["sessions"].sort_values(ascending=False)
    Wsum = W.sum()
    cumsum = 0
    for wthresh in W:
        if cumsum > Wsum * p:
            break
        cumsum += wthresh
    return wthresh

"""
TODO: 2021-05-20
  - test fitting on general traffic sources?
  - test clustering on multiple days of rps data - 
      or rolling rps data or something
  - try out more granular aggs
  - is there some kind of metric which measures total variance
      and computes how much of that variance is captured by a split?

Trevor: 2021-05-21
TODO:
- figure out how to minimize campaign # when writing back to taboola
    - want to make sure campaigns have sufficient traffic
- kw=(location,os,device)
- calc 30-day rps/kw
- can create distribution over the rps(kw) distribution
- 100 campaigns - 1 per percentile

TODO: 2021-05-24
- look into clustering each individual split variable
"""

# split2aggrps = {}
# for split,split_cols in campaign_split_fields.items():
#     print(split,split_cols)
#     rps_df = agg_rps(NOW-90*DAY,NOW,None,traffic_source=TABOOLA,agg_columns=split_cols)
#     rps_df = translate_taboola_vals(rps_df)
#     rps_df["split_on"] = split
#     split2aggrps[split] = rps_df
#     print(split,rps_df.shape)


split2fitaggrps = {}
for split,split_cols in campaign_split_fields.items():
    print(split,split_cols)
    rps_df = agg_rps(NOW-90*DAY,NOW-30*DAY,None,traffic_source=TABOOLA,agg_columns=tuple(split_cols))
    rps_df = translate_taboola_vals(rps_df)
    rps_df["split_on"] = split
    split2fitaggrps[split] = rps_df

    print(split,rps_df.shape)

split2evalaggrps = {}
for split,split_cols in campaign_split_fields.items():
    print(split,split_cols)
    rps_df = agg_rps(NOW-30*DAY,NOW,None,traffic_source=TABOOLA,agg_columns=tuple(split_cols))
    rps_df = translate_taboola_vals(rps_df)
    rps_df["split_on"] = split
    split2evalaggrps[split] = rps_df

    print(split,rps_df.shape)

In [ ]:
import sklearn.cluster
import numpy as np
import scipy.stats
from matplotlib import pyplot as plt
def cluster_split(split):
    rps_df_fit = split2fitaggrps[split]
    rps_df_eval = split2evalaggrps[split]
    split_size = rps_df_fit.__len__()
    MINCLUST = 8
    if split_size <= MINCLUST:
        rps_clust_df = rps_df_eval
        rps_df_fit["clust"] = 0
        rps_df_eval["clust"] = 0
    else:
        # nclust = MINCLUST
        nclust = max(MINCLUST, int(np.log(split_size)))
        # nclust = max(MINCLUST,int(split_size ** 0.5))
        # print("nclust", nclust, split_size, np.log(split_size))
        clusterer = sklearn.cluster.KMeans(n_clusters=nclust)
        rps_df_fit["clust"] = clusterer \
            .fit_predict(
                rps_df_fit["rps_avg"].values.reshape(-1, 1),
                sample_weight=rps_df_fit["sessions"])
        rps_df_eval["clust"] = clusterer \
            .predict(
                rps_df_eval["rps_avg"].values.reshape(-1, 1),
                sample_weight=rps_df_eval["sessions"])
        rps_clust_df = rps_df_eval \
            .groupby("clust") \
            .agg({"rps_avg": get_wavg_by(rps_df_eval, "sessions"), "sessions": sum})
    assert rps_clust_df["rps_avg"].max() <= rps_df_eval["rps_avg"].max()
    rps_wavg = wavg(rps_df_eval["rps_avg"], rps_df_eval["sessions"])
    rps_clust_wavg = wavg(rps_clust_df["rps_avg"], rps_clust_df["sessions"])
    assert abs(rps_wavg - rps_clust_wavg) < 1e-4, (rps_wavg, rps_clust_wavg)
    return rps_df_fit,rps_df_eval,rps_clust_df

def get_split_factor(rps_df):
    orig_index = rps_df.index.names
    split_attr2unique_vals = {index_col: rps_df.index.unique(
        index_col) for index_col in orig_index}
    _,new_index_order = zip(*sorted((V.__len__(),c) for c,V in split_attr2unique_vals.items()))
    nclusts = rps_df["clust"].unique().__len__()
    split_factor = nclusts * np.prod([1] + [split_attr2unique_vals[c].__len__() for c in new_index_order[:-1]])
    return split_factor

perfD = []
for split in campaign_split_fields.keys():
    rps_df_fit,rps_df_eval,rps_clust_df = cluster_split(split)
    perfd = {
        "split": split,
        "fit_shape": rps_df_fit.shape,
        "clust_shape": rps_clust_df.shape,
        # wavg(rps_df["rps_avg"],rps_df["sessions"]),
        "split_variance": wstd(rps_df_eval["rps_avg"], rps_df_eval["sessions"]),
        "cluster_variance": wstd(rps_clust_df["rps_avg"], rps_clust_df["sessions"]),
        # wstd(rps_df["rps_avg"],rps_df["sessions"])
        "split_factor": rps_df_fit.__len__(),
        "clustered_split_factor": get_split_factor(rps_df_fit),
    }
    perfD.append(perfd)
    pprint.pprint(perfd)
    ipydisp(rps_clust_df)

perfdf = pd.DataFrame(perfD)
ipydisp(perfdf)

In [ ]:
# BEST_SPLIT = "location_os_device"
BEST_SPLIT = "dma_os_device"
# BEST_SPLIT = "dma_os"

rps_df_fit, rps_df_eval, rps_clust_df = cluster_split(BEST_SPLIT)
rps_df = rps_df_fit
get_split_factor(rps_df)
orig_index = rps_df.index.names
split_attr2unique_vals = {index_col: rps_df.index.unique(index_col) for index_col in orig_index}
_,new_index_order = zip(*sorted((V.__len__(),c) for c,V in split_attr2unique_vals.items()))
rps_df = rps_df .reset_index()
campaign_df = rps_df \
    .groupby([*new_index_order[:-1], "clust"]) \
    .agg({
        "sessions": sum,
        "rps_avg": get_wavg_by(rps_df,"sessions"),
        new_index_order[-1]: tuple
    })
assert campaign_df["sessions"].sum() == rps_df_fit["sessions"].sum()
camp_rps_wavg = wavg(campaign_df["rps_avg"],campaign_df["sessions"])
fit_rps_wavg = wavg(rps_df_fit["rps_avg"], rps_df_fit["sessions"])
assert abs(camp_rps_wavg - fit_rps_wavg) < 1e-5

excl_campaign_df = campaign_df.groupby([*new_index_order[:-1]]) \
    .agg({
        new_index_order[-1]: tuple
    })
def flatten(M):
    return tuple(el for r in M for el in r)
excl_campaign_df[new_index_order[-1]] = excl_campaign_df[new_index_order[-1]] .apply(flatten)

camps = []
for idx,r in campaign_df.iterrows():
    camp = {
        "sessions_60d": r["sessions"], 
        "rps_avg_60d": r["rps_avg"]
    }
    for field,val in zip(new_index_order[:-1],idx):
        camp[field] = {"includes": val}
    last_field = new_index_order[-1]
    camp[last_field] = {
        "includes": r[last_field]
    }
    camps.append(camp)

for idx, r in excl_campaign_df.iterrows():
    camp = {}
    for field, val in zip(new_index_order[:-1], idx):
        camp[field] = {"includes": val}
    last_field = new_index_order[-1]
    camp[last_field] = {
        "excludes": r[last_field]
    }
    camps.append(camp)

camp_df = pd.DataFrame(camps)
camp_df.to_csv("campaign_dump.csv")